In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

navegador = webdriver.Chrome()
navegador.get("https://web.whatsapp.com")

# esperar a tela do whatsapp carregar
while len(navegador.find_elements(By.ID, 'side')) < 1: # -> lista for vazia -> que o elemento não existe ainda
    time.sleep(1)
time.sleep(2) # só uma garantia

In [ ]:
# o whatsapp já carregou
import pandas as pd

tabela = pd.read_excel("Prof convidado.xlsx")
tabela.columns = tabela.columns.str.strip()
display(tabela[['Nome',
                'Unidade 1', 'Unidade 2', 'Unidade 3', 'Unidade 4', 'Unidade 5',
                'Nome do Curso 1', 'Nome do Curso 2', 'Nome do Curso 3', 'Nome do Curso 4', 'Nome do Curso 5',
                'Data do Curso 1', 'Data do Curso 2', 'Data do Curso 3', 'Data do Curso 4', 'Data do Curso 5',
                'Coordenador 1', 'Coordenador 2', 'Coordenador 3', 'Coordenador 4', 'Coordenador 5',
                'Mensagem',
                'Link de Template', 'Link de Cronograma 1', 'Link de Cronograma 2', 'Link de Cronograma 3', 'Link de Cronograma 4', 'Link de Cronograma 5',
                'Telefone']]) # tem também uma coluna telefone dentro da tabela

In [49]:
import urllib
import time
import os
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import traceback

# Substitua 'tabela' e 'navegador' pelas suas próprias variáveis se ainda não estiverem definidas
# tabela = pd.read_excel('seu_arquivo.xlsx')
# navegador = webdriver.Chrome()  # Exemplo de inicialização

wait = WebDriverWait(navegador, 20)

for linha in tabela.index:
    nome = str(tabela.loc[linha, "Nome"]).title()
    mensagem = tabela.loc[linha, "Mensagem"]
    telefone = tabela.loc[linha, "Telefone"]

    arquivos = [
        tabela.loc[linha, "Link de Template"],
        tabela.loc[linha, "Link de Cronograma 1"],
        tabela.loc[linha, "Link de Cronograma 2"],
        tabela.loc[linha, "Link de Cronograma 3"],
        tabela.loc[linha, "Link de Cronograma 4"],
        tabela.loc[linha, "Link de Cronograma 5"]
    ]

    # Construção dos blocos de curso formatados
    blocos = ""
    for i in range(5):
        unidade = tabela.loc[linha, f"Unidade {i+1}"]
        curso = str(tabela.loc[linha, f"Nome do Curso {i+1}"]).title()
        data = tabela.loc[linha, f"Data do Curso {i+1}"]
        coordenador = tabela.loc[linha, f"Coordenador {i+1}"] if f"Coordenador {i+1}" in tabela.columns else "N"

        if all(pd.notna(x) and x != "N" for x in [unidade, curso, data]):
            # Conversão da data para o formato desejado
            try:
                if isinstance(data, str):
                    data_obj = pd.to_datetime(data)
                else:
                    data_obj = pd.to_datetime(str(data))
                dias_semana = {
                    'Monday': 'Segunda-feira',
                    'Tuesday': 'Terça-feira',
                    'Wednesday': 'Quarta-feira',
                    'Thursday': 'Quinta-feira',
                    'Friday': 'Sexta-feira',
                    'Saturday': 'Sábado',
                    'Sunday': 'Domingo'
                }
                dia_semana_pt = dias_semana.get(data_obj.strftime('%A'), data_obj.strftime('%A'))
                data_formatada = data_obj.strftime(f"%d/%m/%Y ({dia_semana_pt})")
            except Exception as e:
                print(f"Erro ao formatar a data do curso {i+1}: {e}")
                data_formatada = str(data)

            bloco = (
                f"📚 *Curso {i+1}*\n"
                f"📍 *Unidade:* {unidade}\n"
                f"📘 *Nome:* {curso}\n"
                f"🗓 *Data Inicial:* {data_formatada}"
            )
            if pd.notna(coordenador) and coordenador != "N":
                bloco += f"\n👤 *Coordenador:* {coordenador}"
            blocos += bloco + "\n\n"

    blocos = blocos.strip()

    # Montagem da mensagem
    if pd.notna(mensagem):
        texto = str(mensagem).replace("fulano", nome).replace("[BLOCOS_DOS_CURSOS]", blocos)
    else:
        texto = f"Olá {nome}, estamos entrando em contato com você."

    # Adiciona links se houver
    descricoes_arquivos = [
        "Link de Template", "Link de Cronograma 1", "Link de Cronograma 2",
        "Link de Cronograma 3", "Link de Cronograma 4", "Link de Cronograma 5"
    ]
    for desc, arquivo in zip(descricoes_arquivos, arquivos):
        if arquivo != "N":
            texto += f"\n{desc}: {arquivo}"

    # Envio via WhatsApp Web
    texto_encoded = urllib.parse.quote(texto)
    link = f"https://web.whatsapp.com/send?phone={telefone}&text={texto_encoded}"
    navegador.get(link)

    # Aguarda a página carregar
    while len(navegador.find_elements(By.ID, 'side')) < 1:
        time.sleep(1)
    time.sleep(2)

    # Envia a mensagem
    if len(navegador.find_elements(By.XPATH, '//*[@id="app"]/div/span[2]/div/span/div/div/div/div/div/div[1]')) < 1:
        try:
            botao_enviar = wait.until(
                EC.element_to_be_clickable(
                    (By.XPATH, '/html/body/div[1]/div/div/div[3]/div/div[4]/div/footer/div[1]/div/span/div/div[2]/div[2]/button')
                )
            )
            botao_enviar.click()

            for arquivo in arquivos:
                if arquivo != "N" and not arquivo.startswith("http"):
                    caminho_completo = os.path.abspath(f"arquivos/{arquivo}")
                    if os.path.exists(caminho_completo):
                        navegador.find_element(
                            By.XPATH, '//*[@id="main"]/footer/div[1]/div/span/div/div[1]/div/button'
                        ).click()
                        file_input = navegador.find_element(By.CSS_SELECTOR, 'input[type="file"]')
                        file_input.send_keys(caminho_completo)
                        time.sleep(10)
                        navegador.find_element(
                            By.XPATH, '//*[@id="app"]/div/div[3]/div/div[2]/div[2]/span/div/div/div/div[2]/div/div[2]/div[2]/div/div'
                        ).click()
                        time.sleep(10)
                    else:
                        print(f"Arquivo não encontrado: {caminho_completo}")
        except Exception as e:
            print(f"Erro ao enviar mensagem para {nome}: {e}")
            traceback.print_exc()

    print(f"Mensagem enviada para {nome}. Aguardando 15 segundos antes de continuar...")
    time.sleep(15)


Mensagem enviada para Dr.Daniele Luminoso. Aguardando 15 segundos antes de continuar...
Mensagem enviada para Dra. Mariana Zamith . Aguardando 15 segundos antes de continuar...
